In [3]:
import pandas as pd
import numpy as np

# --------------------------
# 1. 读取数据（适配你的数据路径）
# --------------------------
def read_data(data_path,start_date,end_date):
    """读取Parquet数据，并检查核心字段是否存在"""
    # 读取数据
    try:
        df = pd.read_parquet(data_path)
        df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
        print(f"✅ 数据读取成功！全量数据行数：{len(df)}")
    except Exception as e:
        print(f"❌ 数据读取失败：{str(e)}")
        print("请检查数据路径是否正确，或文件是否为Parquet格式！")
        return None

    return df
    

In [5]:
DATA_PATH = r'D:\workspace\xiaoyao\data\factortable.parquet'

# 步骤1：读取数据并检查
df = read_data(DATA_PATH,'2025-01-01','2025-06-30')
df.head(3)

✅ 数据读取成功！全量数据行数：601324


,date,stock_code,open,close,low,high,volume,money,factor,high_limit,...,order_book_volume_ratio,obv_ratio_vs_yesterday,obv_ratio_vs_5d_avg,turnover_ratio_vs_yesterday,turnover_ratio_vs_5d_avg,money_ratio_vs_yesterday,money_ratio_vs_5d_avg,amplitude,amplitude_vs_yesterday,amplitude_vs_5d_avg
0,2025-01-02,000001.XSHE,1630.12,1588.43,1582.87,1635.68,1309344.0,2.102923e+09,138.970157,1788.55,...,1.106891,NaN,NaN,NaN,NaN,NaN,NaN,3.247947,NaN,NaN
1,2025-01-03,000001.XSHE,1589.82,1581.48,1578.70,1603.72,830884.0,1.320521e+09,138.970157,1746.85,...,2.388428,2.157780,2.157780,0.634531,0.634531,0.627945,0.627945,1.575140,0.484965,0.484965
2,2025-01-06,000001.XSHE,1581.48,1589.82,1559.25,1595.38,781129.0,1.234306e+09,138.970157,1739.91,...,6.639049,2.779673,3.798823,0.940168,0.729954,0.934711,0.721090,2.284569,1.450391,0.947347


In [7]:
# 2. 基础数据清洗（剔除无效数据）
# --------------------------
def clean_basic_data(df):
    """基础清洗：剔除停牌、无收益、无成交量的无效样本"""
    df_clean = df.copy()
    
    # 1. 日期格式化（统一为datetime类型，便于后续时间筛选）
    df_clean["date"] = pd.to_datetime(df_clean["date"])
    
    # 2. 剔除无效样本（核心过滤条件）
    filter_conditions = [
        (df_clean["return_1d"].notna()),
        (df_clean["money_ratio_vs_5d_avg"].notna()),
        (df_clean["return_1d"] > 0),          # 只保留正收益样本
        (df_clean["return_1d"] <= 0.5),       # 排除单日收益>50%的极端值
        (df_clean["volume"] > 0)              # 有成交量
    ]
    
    # 若数据包含"paused"字段，加入停牌过滤
    if "paused" in df_clean.columns:
        filter_conditions.append(df_clean["paused"] == 0)
        print("⚠️  已加入停牌过滤（paused=0）")
    
    # 应用过滤条件
    df_clean = df_clean[np.all(filter_conditions, axis=0)].reset_index(drop=True)
    
    # 输出清洗结果
    print(f"\n📊 数据清洗结果：")
    print(f"- 清洗前总样本数：{len(df)}")
    print(f"- 清洗后有效样本数：{len(df_clean)}")
    print(f"- 有效样本时间范围：{df_clean['date'].min().strftime('%Y-%m-%d')} ~ {df_clean['date'].max().strftime('%Y-%m-%d')}")
    print(f"- 有效样本平均次日收益：{df_clean['return_1d'].mean():.2%}")
    
    return df_clean

In [8]:
df_clean = clean_basic_data(df)
df_clean.head(3) 

⚠️  已加入停牌过滤（paused=0）

📊 数据清洗结果：
- 清洗前总样本数：601324
- 清洗后有效样本数：308926
- 有效样本时间范围：2025-01-03 ~ 2025-06-30
- 有效样本平均次日收益：3.12%


,date,stock_code,open,close,low,high,volume,money,factor,high_limit,...,order_book_volume_ratio,obv_ratio_vs_yesterday,obv_ratio_vs_5d_avg,turnover_ratio_vs_yesterday,turnover_ratio_vs_5d_avg,money_ratio_vs_yesterday,money_ratio_vs_5d_avg,amplitude,amplitude_vs_yesterday,amplitude_vs_5d_avg
0,2025-01-03,000001.XSHE,1589.82,1581.48,1578.70,1603.72,830884.0,1.320521e+09,138.970157,1746.85,...,2.388428,2.157780,2.157780,0.634531,0.634531,0.627945,0.627945,1.575140,0.484965,0.484965
1,2025-01-06,000001.XSHE,1581.48,1589.82,1559.25,1595.38,781129.0,1.234306e+09,138.970157,1739.91,...,6.639049,2.779673,3.798823,0.940168,0.729954,0.934711,0.721090,2.284569,1.450391,0.947347
2,2025-01-10,000001.XSHE,1584.26,1570.36,1567.58,1592.60,574320.0,9.050050e+08,138.970157,1742.69,...,1.480146,3.272096,0.653240,1.061967,0.831043,1.054986,0.823542,1.579286,1.210529,0.922341


In [12]:
import pandas as pd
import numpy as np

# --------------------------
# 3. 筛选高收益率样本（按每日分位数筛选，每日前20%）
# --------------------------
def select_daily_high_return_samples(df_clean, top_pct=20):
    """
    修正版：按日期分组，每日筛选return_1d前N%的个股（默认前20%）
    逻辑：每天单独计算分位数，确保每日都有高收益样本，而非全量数据筛选
    """
    df_clean_copy = df_clean.copy()
    
    # 关键步骤：按日期分组，计算每日的高收益阈值（当日return_1d的前20%分位数）
    # 1 - top_pct/100：例如前20% → 取80分位数（当日收益≥80分位数即为前20%）
    df_clean_copy["daily_high_return_threshold"] = df_clean_copy.groupby("date")["return_1d"].transform(
        lambda x: x.quantile(1 - top_pct / 100)
    )
    
    # 筛选每日前20%的高收益样本
    daily_high_return_df = df_clean_copy[
        df_clean_copy["return_1d"] >= df_clean_copy["daily_high_return_threshold"]
    ].copy()
    
    # 输出整体统计信息
    print(f"\n🎯 每日高收益样本筛选结果（每日前{top_pct}%）：")
    print(f"- 总交易日数：{df_clean_copy['date'].nunique()}天")
    print(f"- 高收益样本总数：{len(daily_high_return_df)}")
    print(f"- 日均高收益样本数：{len(daily_high_return_df)/df_clean_copy['date'].nunique():.1f}只")
    print(f"- 所有高收益样本平均次日收益：{daily_high_return_df['return_1d'].mean():.2%}")
    print(f"- 所有高收益样本最高次日收益：{daily_high_return_df['return_1d'].max():.2%}")
    
    # 输出每日筛选详情（前10天）
    print(f"\n📅 每日高收益样本详情（前10天）：")
    # 按日期统计每日的阈值、样本数、平均收益
    daily_detail = daily_high_return_df.groupby("date").agg({
        "daily_high_return_threshold": "first",  # 当日高收益阈值（所有样本相同）
        "stock_code": "count",                  # 当日高收益样本数
        "return_1d": "mean"                     # 当日高收益样本平均收益
    }).reset_index()
    # 重命名列名，便于理解
    daily_detail.columns = ["日期", "当日高收益阈值（≥此收益）", "当日高收益样本数", "当日高收益样本平均收益"]
    # 格式化输出（阈值和平均收益转百分比）
    daily_detail["当日高收益阈值（≥此收益）"] = daily_detail["当日高收益阈值（≥此收益）"].apply(lambda x: f"{x:.2%}")
    daily_detail["当日高收益样本平均收益"] = daily_detail["当日高收益样本平均收益"].apply(lambda x: f"{x:.2%}")
    print(daily_detail.head(10).to_string(index=False))
    
    # 保留核心字段，删除临时计算字段
    final_daily_high_return_df = daily_high_return_df.drop(columns=["daily_high_return_threshold"])
    
    return final_daily_high_return_df, daily_detail

# --------------------------
# 调用示例（替换原有的select_high_return_samples调用）
# --------------------------
if __name__ == "__main__":
    # 假设已完成数据读取和清洗（df_clean为清洗后的有效样本）
    
    # 调用修正版函数，每日筛选前20%高收益个股
    daily_high_return_df, daily_detail = select_daily_high_return_samples(df_clean, top_pct=20)
    
    # 保存结果为Parquet（延续之前的格式）
    daily_high_return_df.to_parquet("第一步_每日高收益样本（每日前20%）.parquet", index=False)
    daily_detail.to_parquet("第一步_每日高收益样本详情.parquet", index=False)
    
    print(f"\n💾 修正版结果已保存：")
    print(f"- 每日高收益个股明细：第一步_每日高收益样本（每日前20%）.parquet")
    print(f"- 每日筛选详情（阈值/样本数）：第一步_每日高收益样本详情.parquet")


🎯 每日高收益样本筛选结果（每日前20%）：
- 总交易日数：116天
- 高收益样本总数：61837
- 日均高收益样本数：533.1只
- 所有高收益样本平均次日收益：8.14%
- 所有高收益样本最高次日收益：48.02%

📅 每日高收益样本详情（前10天）：
        日期 当日高收益阈值（≥此收益）  当日高收益样本数 当日高收益样本平均收益
2025-01-03         4.58%       786       7.80%
2025-01-06         5.12%       716       9.14%
2025-01-07         4.27%       483       8.66%
2025-01-08         5.37%       183      10.45%
2025-01-09         5.09%       154       9.09%
2025-01-10         8.32%       995      10.87%
2025-01-13         5.57%       953       8.69%
2025-01-14         3.21%       455       6.66%
2025-01-15         3.07%       510       6.03%
2025-01-16         3.77%       744       6.80%

💾 修正版结果已保存：
- 每日高收益个股明细：第一步_每日高收益样本（每日前20%）.parquet
- 每日筛选详情（阈值/样本数）：第一步_每日高收益样本详情.parquet


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --------------------------
# 1. 读取每日高收益样本数据
# --------------------------
def read_high_return_data(file_path):
    """读取每日高收益样本数据并检查关键字段"""
    try:
        df = pd.read_parquet(file_path)
        print(f"✅ 成功读取高收益样本数据，共 {len(df)} 条记录")
    except Exception as e:
        print(f"❌ 数据读取失败：{str(e)}")
        return None
    
    # 检查是否包含必要字段
    required_fields = ["date", "stock_code", "close", "ma5", "return_1d"]
    missing_fields = [f for f in required_fields if f not in df.columns]
    
    if missing_fields:
        print(f"❌ 缺失必要字段：{missing_fields}")
        print(f"数据中包含的字段：{list(df.columns)}")
        return None
    
    # 确保日期格式正确
    df["date"] = pd.to_datetime(df["date"])
    return df

# --------------------------
# 2. 计算趋势分类所需指标
# --------------------------
def calculate_trend_indicators(df):
    """计算用于趋势分类的指标"""
    df_copy = df.copy()
    
    # 1. 收盘价与5日均线的比值（判断是否在均线上方）
    df_copy["close_ma5_ratio"] = df_copy["close"] / df_copy["ma5"]
    
    # 2. 5日均线斜率（判断均线趋势方向）
    # 计算方法：(今日MA5 - 昨日MA5) / 昨日MA5
    df_copy = df_copy.sort_values(["stock_code", "date"])
    df_copy["ma5_slope"] = df_copy.groupby("stock_code")["ma5"].transform(
        lambda x: (x - x.shift(1)) / x.shift(1)
    )
    
    # 3. 收盘价在5日均线上方的持续天数
    df_copy["above_ma5_days"] = df_copy.groupby("stock_code").apply(
        lambda group: (group["close"] > group["ma5"]).astype(int).rolling(window=5, min_periods=1).sum()
    ).reset_index(level=0, drop=True)
    
    return df_copy

# --------------------------
# 3. 按趋势类型分类
# --------------------------
def classify_trend_types(df):
    """
    将股票分为三类：
    1. 5日均线之上的上升趋势
    2. 5日均线之下的下跌趋势
    3. 震荡类型
    """
    df_copy = df.copy()
    
    # 定义分类条件
    # 1. 上升趋势：收盘价在5日均线上方 + 5日均线向上 + 最近5天多数时间在均线上方
    rising_condition = (
        (df_copy["close_ma5_ratio"] > 1.01) &  # 收盘价高于5日均线1%以上
        (df_copy["ma5_slope"] > 0.005) &       # 5日均线上行（斜率>0.5%）
        (df_copy["above_ma5_days"] >= 4)       # 最近5天至少4天在均线上方
    )
    
    # 2. 下跌趋势：收盘价在5日均线下方 + 5日均线下行 + 最近5天多数时间在均线下方
    falling_condition = (
        (df_copy["close_ma5_ratio"] < 0.99) &  # 收盘价低于5日均线1%以上
        (df_copy["ma5_slope"] < -0.005) &      # 5日均线下行（斜率<-0.5%）
        (df_copy["above_ma5_days"] <= 1)       # 最近5天最多1天在均线上方
    )
    
    # 3. 震荡类型：不符合上述两种趋势的情况
    df_copy["trend_type"] = np.select(
        [rising_condition, falling_condition],
        ["上升趋势", "下跌趋势"],
        default="震荡类型"
    )
    
    # 统计分类结果
    print("\n📊 趋势类型分类统计：")
    trend_counts = df_copy["trend_type"].value_counts()
    trend_percent = df_copy["trend_type"].value_counts(normalize=True) * 100
    
    for trend_type in ["上升趋势", "下跌趋势", "震荡类型"]:
        count = trend_counts.get(trend_type, 0)
        percent = trend_percent.get(trend_type, 0)
        avg_return = df_copy[df_copy["trend_type"] == trend_type]["return_1d"].mean()
        print(f"- {trend_type}：{count} 只（{percent:.1f}%），平均次日收益：{avg_return:.2%}")
    
    return df_copy

# --------------------------
# 4. 可视化各类趋势的收益分布
# --------------------------
def visualize_trend_returns(df):
    """可视化不同趋势类型的收益分布"""
    plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]
    plt.rcParams["axes.unicode_minus"] = True  # 正确显示负号
    
    # 绘制箱线图
    plt.figure(figsize=(12, 6))
    df.boxplot(column="return_1d", by="trend_type", grid=False, showfliers=False)
    plt.title("不同趋势类型的次日收益分布", fontsize=14)
    plt.suptitle("")  # 去除pandas自动添加的标题
    plt.xlabel("趋势类型", fontsize=12)
    plt.ylabel("次日收益率", fontsize=12)
    plt.axhline(y=0, color="r", linestyle="--", alpha=0.3)
    
    # 将y轴转换为百分比显示
    ax = plt.gca()
    ax.set_yticklabels([f"{x:.0%}" for x in ax.get_yticks()])
    
    plt.tight_layout()
    plt.savefig("不同趋势类型的收益分布.png", dpi=300, bbox_inches="tight")
    plt.close()
    print("\n📈 已生成不同趋势类型的收益分布图：不同趋势类型的收益分布.png")

# --------------------------
# 主函数：完整流程
# --------------------------
def main():
    # 读取数据
    high_return_df = read_high_return_data("第一步_每日高收益样本（每日前20%）.parquet")
    if high_return_df is None:
        return
    
    # 计算趋势指标
    high_return_with_indicators = calculate_trend_indicators(high_return_df)
    
    # 分类趋势类型
    high_return_with_trend = classify_trend_types(high_return_with_indicators)
    
    # 可视化收益分布
    visualize_trend_returns(high_return_with_trend)
    
    # 保存分类结果
    output_cols = ["date", "stock_code", "close", "ma5", "return_1d", 
                  "close_ma5_ratio", "ma5_slope", "trend_type"]
    high_return_with_trend[output_cols].to_parquet(
        "第二步_高收益样本趋势分类.parquet", 
        index=False
    )
    print("\n💾 已保存趋势分类结果：第二步_高收益样本趋势分类.parquet")

if __name__ == "__main__":
    main()
    